In [9]:
import pandas as pd

In [10]:
from pyspark.sql import SparkSession

In [11]:
from pyspark.sql.functions import col, desc

In [12]:
spark = SparkSession.builder.appName('dataframe').getOrCreate()

In [14]:
data = spark.read.format('csv').\
    option('inferSchema', 'true').\
    option('header', 'true').\
    option('path', 'operations_management.csv').\
    load()

In [15]:
#if option == 'location' instead of 'path', we'll have to provide a custom
#schema to read the file. setting location gives the error-
#AnalysisException: [UNABLE_TO_INFER_SCHEMA] Unable to infer schema for CSV. 
#It must be specified manually.
#here's the code for that-

"""
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define the schema for your CSV data
custom_schema = StructType([
    StructField("description", StringType(), True),  # Replace with your column names and data types
    StructField("industry", StringType(), True),
    StructField("level", IntegerType(), True),
    StructField("size", StringType(), True),
    StructField("line_code", StringType(), True),
    StructField("value", IntegerType(), True),

])

# Replace 'your_data.csv' with the path to your CSV file
data = spark.read.csv("operations_management.csv", schema=custom_schema, header=True)

# Now you can work with your data
data.show()
"""

'\nfrom pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType\n\n# Define the schema for your CSV data\ncustom_schema = StructType([\n    StructField("description", StringType(), True),  # Replace with your column names and data types\n    StructField("industry", StringType(), True),\n    StructField("level", IntegerType(), True),\n    StructField("size", StringType(), True),\n    StructField("line_code", StringType(), True),\n    StructField("value", IntegerType(), True),\n\n])\n\n# Replace \'your_data.csv\' with the path to your CSV file\ndata = spark.read.csv("operations_management.csv", schema=custom_schema, header=True)\n\n# Now you can work with your data\ndata.show()\n'

In [16]:
data.show()

+--------------------+--------------------+-----+---------------+---------+-----+
|         description|            industry|level|           size|line_code|value|
+--------------------+--------------------+-----+---------------+---------+-----+
|Awareness of clim...|               total|    0| 6–19 employees| C0300.01|13080|
|Awareness of clim...|               total|    0|20–49 employees| C0300.01| 3348|
|Awareness of clim...|               total|    0|50–99 employees| C0300.01| 1089|
|Awareness of clim...|               total|    0| 100+ employees| C0300.01| 1023|
|Awareness of clim...|Agriculture, fore...|    1|          total| C0300.01| 2364|
|Awareness of clim...|         Agriculture|    2|          total| C0300.01| 1683|
|Awareness of clim...|  Commercial fishing|    2|          total| C0300.01|   27|
|Awareness of clim...|  Forestry & logging|    2|          total| C0300.01|  126|
|Awareness of clim...|Agriculture, fore...|    2|          total| C0300.01|  528|
|Awareness of cl

In [17]:
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: integer (nullable = true)



In [18]:
data_1 = data.select('industry', 'value').\
filter((col('value')>200) & (col('industry')!='total')).\
orderBy(desc('value'))

In [19]:
data_1.show(5)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [20]:
data_1.printSchema()

root
 |-- industry: string (nullable = true)
 |-- value: integer (nullable = true)



In [21]:
data_2 = data.select('industry', 'value').\
filter((col('value')<30) & (col('industry')=='Auxiliary')).\
orderBy('value')

In [22]:
data_2.show(10)

+---------+-----+
| industry|value|
+---------+-----+
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    3|
|Auxiliary|    3|
|Auxiliary|    3|
+---------+-----+
only showing top 10 rows



In [26]:
#session scoped - temporary dataframe
#dont give extension here
data.createOrReplaceTempView("operations_management")

In [27]:
data_3 = spark.sql("""
select industry, value from operations_management
where industry = 'Auxiliary' and value < 30
order by value;
""")

In [28]:
data_3.show(10)

+---------+-----+
| industry|value|
+---------+-----+
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    0|
|Auxiliary|    3|
|Auxiliary|    3|
|Auxiliary|    3|
+---------+-----+
only showing top 10 rows



In [34]:
data_4 = spark.sql("""
select industry, value from operations_management
where industry != 'total' and value > 200
order by value desc;
""")

In [35]:
data_4.show(5)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [36]:
data_4.printSchema()

root
 |-- industry: string (nullable = true)
 |-- value: integer (nullable = true)

